In [1]:
# import 
import pandas as pd;
import numpy as np;

In [5]:
# Datasets
X1 = pd.read_csv('../datasets/complete-data/Caltech/GISTFeature.csv',header=None);
X2 = pd.read_csv('../datasets/complete-data/Caltech/LBPFeature.csv',header=None);
print(X1.shape) # (1474, 512)
print(X2.shape) # (1474,928)
X1.head()

(1474, 512)
(1474, 928)


0         1         2         3         4         5         6    \
0  0.149860  0.049631  0.049833  0.030009  0.166100  0.045195  0.052351   
1  0.168870  0.082086  0.167050  0.109790  0.113190  0.045916  0.050687   
2  0.031449  0.038602  0.040515  0.064938  0.024486  0.064542  0.038623   
3  0.062685  0.069544  0.061907  0.038404  0.052422  0.041499  0.053826   
4  0.062920  0.034458  0.015966  0.022567  0.127380  0.098888  0.062020   

        7         8         9    ...       502       503       504       505  \
0  0.032841  0.072760  0.067054  ...  0.045340  0.038174  0.068873  0.088719   
1  0.031878  0.207420  0.102760  ...  0.118360  0.075107  0.081991  0.046442   
2  0.055522  0.035708  0.034724  ...  0.040049  0.055716  0.054746  0.068624   
3  0.048148  0.034058  0.040677  ...  0.085608  0.071643  0.119500  0.058825   
4  0.031535  0.049430  0.058208  ...  0.029204  0.038763  0.047977  0.071451   

        506       507       508       509       510       511  
0  0.060006  0.046778  0.024452  0.050038  0.096785  0.070728  
1  0.163500  0.110060  0.112140  0.085244  0.079044  0.051326  
2  0.043013  0.089594  0.009888  0.060689  0.055650  0.049560  
3  0.082927  0.062791  0.048369  0.044633  0.069152  0.102740  
4  0.072366  0.055249  0.028656  0.041247  0.110050  0.054509  

[5 rows x 512 columns]

In [54]:
# Normalize
from sklearn.preprocessing import Normalizer
transformer = Normalizer();
Xn1 = transformer.transform(X1).T
Xn2 = transformer.transform(X2).T

In [55]:
np.random.seed(123)
import numpy.matlib
Ht = np.matlib.rand(V[0].shape)

In [56]:
V = [Xn1,Xn2];

In [68]:
V[0].shape  # 10x 1474
# H => 10 x 1474
# w => 1474 x 10


(512, 1474)

In [ ]:
# 512 x 1474 => (512 x 10) x (10 x 1474)
#      X     =>   W      x   H

In [182]:
# random H1,H2, W1,W2 # k = 5
H=[]
W=[]
for i in range(len(V)):
    sh = V[i].shape
    Wt = np.matlib.rand(sh[0],10)
    Ht = np.matlib.rand(sh[1],10).T
    W.append(Wt);
    H.append(Ht);
    

In [170]:
print(W[0].shape)
print(H[0].shape)

(512, 10)
(10, 1474)


In [152]:
# update H
# 1474 x something

# W = (512 x 10)
# X = (512 x 1474)
# HF = (10 x 1474)
# H = list [10 x 1470][10 x 1470]
# view =  0

# up => (10 x 512 )(512 x 1474) => (10 x 1474)
# mid => (10  x 14704 )
# other => (10 x 512)(512x 10)(10 x 1470) => (10 x 1474)
# mid + other > (10 x 1470)
# final > (10 x 1470)


def h_coefficient_matrix(W,X,HF,alpha,beta,H,view_no):
    # Upper calculation
    up= 2*(W.T)*X # 512 x 1474 * 1474 x 512 => 512 x 512
    # for middle loop
    mid_H=0;
    for i in range(len(H)): # [H0, H1]
        if(i == view_no):
            continue;
        mid_H = mid_H + H[i];
    mid_H = alpha*mid_H;
    down = (2*W.T*(W)*HF)+2*beta*HF+mid_H; # H => previous H
    final = up/down;
    return final;

In [151]:
# W = (512 x 10)
# X = (512 x 1474)
# HF = (10 x 1474)
# H = list [10 x 1470][10 x 1470]
# view =  0

# up => 512x1474 ) (1474 x 10 ) => 512 x 10
# dow => 512 x 10 )( 10x1474 ) (1474 x 10) => 512 x 10

def w_cofficient_matrix(X,H,W):
    up = X * (H.T);
    down = W * H * (H.T)
    final = up/down
    return final;
    
    

In [145]:
from sklearn.metrics.pairwise import euclidean_distances
x = np.array([[1,2],[3,4.5]])
y = np.array([[1,2.1],[3,4]])
d = euclidean_distances(x,y)
di = [];
for i in range(2):
    di.append(d[i][i])
di
max(di)

0.5

In [124]:
a = V[0].shape
a[0]

512

In [164]:
np.multiply(H[0],H[1]).shape

(10, 1474)

In [175]:
# W = (512 x 10)
# X = (512 x 1474)
# HF = (10 x 1474)

def findError(X,W,H):
    loop = X.shape
    loop = loop[1];
    sumz = 0;
    L =W*H ; # 512 x 10 10x 1474 => 512 x 1474
    L = L.T;
    X = X.T;
    error = euclidean_distances(X,L);
    DI= [];
    for i in range(loop):
        DI.append(error[i][i])
        
    return max(DI),np.average(DI);

In [184]:
alpha = 0.0001;
beta = 0.0001;
iteration = 0;

for index,X in enumerate(V): # X1, X2
    max_E , avg_E = findError(X,W[index],H[index]);
    while( max_E >= 0.566): # 0.5 
        # updata H 
        H[index] =  np.multiply(H[index],h_coefficient_matrix(W[index],X,H[index],alpha,beta,H,index));
        # update W
        W[index] =  np.multiply(W[index],w_cofficient_matrix(X,H[index],W[index]));
        # increase number of iteration
        iteration = iteration + 1;
        # display info
        max_E , avg_E = findError(X,W[index],H[index]);
        print(f"Error:: of Iteration {iteration} => ","Maximum Error: ",max_E," &&& Average Error:",avg_E);
        # End of View or End of While Loop;
    # inside For Loop for View
    print('End of View ',index)
    

End of View  0
Error:: of Iteration 1 =>  Maximum Error:  0.751517807097332  &&& Average Error: 0.35438022162617594
Error:: of Iteration 2 =>  Maximum Error:  0.6979994206838757  &&& Average Error: 0.33770054822138745
Error:: of Iteration 3 =>  Maximum Error:  0.6939661073771007  &&& Average Error: 0.3355179157595168
Error:: of Iteration 4 =>  Maximum Error:  0.6899264035419238  &&& Average Error: 0.3333945430288152
Error:: of Iteration 5 =>  Maximum Error:  0.6849025416893237  &&& Average Error: 0.3310956330389493
Error:: of Iteration 6 =>  Maximum Error:  0.6784549063826059  &&& Average Error: 0.3284326542644478
Error:: of Iteration 7 =>  Maximum Error:  0.6701242122794869  &&& Average Error: 0.3251911503218026
Error:: of Iteration 8 =>  Maximum Error:  0.6594635979756991  &&& Average Error: 0.32113802383597395
Error:: of Iteration 9 =>  Maximum Error:  0.6461736408030081  &&& Average Error: 0.3160565964373361
Error:: of Iteration 10 =>  Maximum Error:  0.6303215231479009  &&& Averag

In [213]:
# find avearage of H => H*

# initialize all to 0 with H dimenstion
sum_H = np.zeros(H[0].shape);

for h in H:
    sum_H = sum_H + h;

# Average of all H ;
H_star = sum_H/len(H);


matrix([[0.00865075, 0.00817312, 0.00267336, ..., 0.00063916, 0.00151358,
         0.00358691],
        [0.01105359, 0.00723174, 0.01081549, ..., 0.00418431, 0.0073252 ,
         0.00718541],
        [0.00688953, 0.01228485, 0.00795208, ..., 0.00669529, 0.0059402 ,
         0.0021677 ]])

## Overall Coding
1. Load Data
2. Normalize
3. Randomize H and W
4. Find H and W
5. Averaging H

> input => multiview dataset V, alpha , beta;

> output => H*

In [214]:
# 0. Import All requirements
import pandas as pd;
import numpy as np;
import numpy.matlib;
from sklearn.preprocessing import Normalizer

In [216]:
# 1 & 2. Load Data and Normalize
#
# LOAD
# 
X1 = pd.read_csv('../datasets/complete-data/Caltech/GISTFeature.csv',header=None);
X2 = pd.read_csv('../datasets/complete-data/Caltech/LBPFeature.csv',header=None);
# NORMALIZE
#
transformer = Normalizer();
Xn1 = transformer.transform(X1).T
Xn2 = transformer.transform(X2).T
# COMBINE
#
V = [Xn1,Xn2];

In [221]:
# 3. Random value of H and W for all View
num_of_dimension = 10;  # Change this to test with other dimension :
H=[]
W=[]
for i in range(len(V)):
    sh = V[i].shape
    Wt = np.matlib.rand(sh[0],num_of_dimension)
    Ht = np.matlib.rand(sh[1],num_of_dimension).T
    W.append(Wt);
    H.append(Ht);
# End of H and W Randomization

In [222]:
# 4. Update H and W for all View
alpha = 0.0001; # parameter
beta = 0.0001;  # parameter 
iteration = 0;  # tracking of iteration

for index,X in enumerate(V): # X1, X2
    max_E , avg_E = findError(X,W[index],H[index]);
    while( max_E >= 0.566): # Effect by random value of H , W ;
        # updata H 
        H[index] =  np.multiply(H[index],h_coefficient_matrix(W[index],X,H[index],alpha,beta,H,index));
        # update W
        W[index] =  np.multiply(W[index],w_cofficient_matrix(X,H[index],W[index]));
        # increase number of iteration
        iteration = iteration + 1;
        # display info
        max_E , avg_E = findError(X,W[index],H[index]);
        print(f"Error:: of Iteration {iteration} => ","Maximum Error: ",max_E," &&& Average Error:",avg_E);
        # End of View or End of While Loop;
    # inside For Loop for View
    print('End of View ',index)

Error:: of Iteration 1 =>  Maximum Error:  0.8635777136252809  &&& Average Error: 0.4527347925591284
Error:: of Iteration 2 =>  Maximum Error:  0.8593151677155911  &&& Average Error: 0.44744688753887474
Error:: of Iteration 3 =>  Maximum Error:  0.8584057614847014  &&& Average Error: 0.4461484750914414
Error:: of Iteration 4 =>  Maximum Error:  0.8575091420741975  &&& Average Error: 0.4449346211865375
Error:: of Iteration 5 =>  Maximum Error:  0.8566337154091139  &&& Average Error: 0.4437063440727291
Error:: of Iteration 6 =>  Maximum Error:  0.855741023943003  &&& Average Error: 0.4423819251121379
Error:: of Iteration 7 =>  Maximum Error:  0.8547904836619654  &&& Average Error: 0.4408626546657914
Error:: of Iteration 8 =>  Maximum Error:  0.8537367229984156  &&& Average Error: 0.4390230324567163
Error:: of Iteration 9 =>  Maximum Error:  0.8525237903068932  &&& Average Error: 0.4367041457024183
Error:: of Iteration 10 =>  Maximum Error:  0.8510790208188479  &&& Average Error: 0.433714

Error:: of Iteration 82 =>  Maximum Error:  0.7635410769816254  &&& Average Error: 0.3032740676493123
Error:: of Iteration 83 =>  Maximum Error:  0.7637788303497761  &&& Average Error: 0.3030116611578179
Error:: of Iteration 84 =>  Maximum Error:  0.7639415781375982  &&& Average Error: 0.30275979130621855
Error:: of Iteration 85 =>  Maximum Error:  0.7640277928977722  &&& Average Error: 0.30251790088326774
Error:: of Iteration 86 =>  Maximum Error:  0.7640361408730322  &&& Average Error: 0.3022854613515282
Error:: of Iteration 87 =>  Maximum Error:  0.7639655176678691  &&& Average Error: 0.30206197252915007
Error:: of Iteration 88 =>  Maximum Error:  0.7638150997478058  &&& Average Error: 0.30184696170520936
Error:: of Iteration 89 =>  Maximum Error:  0.7635844067872641  &&& Average Error: 0.3016399825059373
Error:: of Iteration 90 =>  Maximum Error:  0.7632733659297372  &&& Average Error: 0.3014406137254754
Error:: of Iteration 91 =>  Maximum Error:  0.7628823644726648  &&& Average Er

Error:: of Iteration 163 =>  Maximum Error:  0.673618330349897  &&& Average Error: 0.2950172059661007
Error:: of Iteration 164 =>  Maximum Error:  0.6727506087026988  &&& Average Error: 0.29497468727279264
Error:: of Iteration 165 =>  Maximum Error:  0.6718940204301134  &&& Average Error: 0.29493273515990365
Error:: of Iteration 166 =>  Maximum Error:  0.6710483862415164  &&& Average Error: 0.29489133416699936
Error:: of Iteration 167 =>  Maximum Error:  0.6702135195206298  &&& Average Error: 0.2948504698447262
Error:: of Iteration 168 =>  Maximum Error:  0.6693892290096357  &&& Average Error: 0.29481012883430247
Error:: of Iteration 169 =>  Maximum Error:  0.6685753216181152  &&& Average Error: 0.2947702989150276
Error:: of Iteration 170 =>  Maximum Error:  0.6677716052314979  &&& Average Error: 0.29473096901382717
Error:: of Iteration 171 =>  Maximum Error:  0.6669778913887119  &&& Average Error: 0.2946921291722729
Error:: of Iteration 172 =>  Maximum Error:  0.6661939976978104  &&& 

Error:: of Iteration 243 =>  Maximum Error:  0.6287132666995684  &&& Average Error: 0.29281309611311723
Error:: of Iteration 244 =>  Maximum Error:  0.6283751447647719  &&& Average Error: 0.29279696394446425
Error:: of Iteration 245 =>  Maximum Error:  0.628040685145508  &&& Average Error: 0.2927810309726033
Error:: of Iteration 246 =>  Maximum Error:  0.6277097932652489  &&& Average Error: 0.2927652921811135
Error:: of Iteration 247 =>  Maximum Error:  0.6273823737000809  &&& Average Error: 0.29274974249510216
Error:: of Iteration 248 =>  Maximum Error:  0.6270583303321309  &&& Average Error: 0.29273437681155207
Error:: of Iteration 249 =>  Maximum Error:  0.6267375665356486  &&& Average Error: 0.29271919003133023
Error:: of Iteration 250 =>  Maximum Error:  0.6264199854022949  &&& Average Error: 0.2927041770920016
Error:: of Iteration 251 =>  Maximum Error:  0.6261054900101597  &&& Average Error: 0.29268933300052274
Error:: of Iteration 252 =>  Maximum Error:  0.6257939837384723  &&&

Error:: of Iteration 323 =>  Maximum Error:  0.6086900645413662  &&& Average Error: 0.29190257001561803
Error:: of Iteration 324 =>  Maximum Error:  0.6085040100234426  &&& Average Error: 0.29189382502567146
Error:: of Iteration 325 =>  Maximum Error:  0.6083186354317524  &&& Average Error: 0.2918851195070163
Error:: of Iteration 326 =>  Maximum Error:  0.6081339298961475  &&& Average Error: 0.2918764629882102
Error:: of Iteration 327 =>  Maximum Error:  0.6079498933629747  &&& Average Error: 0.29186786555754823
Error:: of Iteration 328 =>  Maximum Error:  0.6077665365183808  &&& Average Error: 0.2918593374394354
Error:: of Iteration 329 =>  Maximum Error:  0.6075838802284103  &&& Average Error: 0.2918508885354897
Error:: of Iteration 330 =>  Maximum Error:  0.6074019544958046  &&& Average Error: 0.291842527963775
Error:: of Iteration 331 =>  Maximum Error:  0.6072207969708566  &&& Average Error: 0.29183426363068987
Error:: of Iteration 332 =>  Maximum Error:  0.6070404510882038  &&& A

...

Error:: of Iteration 652 =>  Maximum Error:  0.5803717508905113  &&& Average Error: 0.2907098083791836
Error:: of Iteration 653 =>  Maximum Error:  0.5803335611449402  &&& Average Error: 0.2907084541695795
Error:: of Iteration 654 =>  Maximum Error:  0.5802954791657503  &&& Average Error: 0.2907071066795891
Error:: of Iteration 655 =>  Maximum Error:  0.5802575034839655  &&& Average Error: 0.2907057660436722
Error:: of Iteration 656 =>  Maximum Error:  0.5802196326178407  &&& Average Error: 0.29070443238974303
Error:: of Iteration 657 =>  Maximum Error:  0.5801818650619529  &&& Average Error: 0.29070310583804276
Error:: of Iteration 658 =>  Maximum Error:  0.5801441992768428  &&& Average Error: 0.2907017865003
Error:: of Iteration 659 =>  Maximum Error:  0.58010663367952  &&& Average Error: 0.2907004744791496
Error:: of Iteration 660 =>  Maximum Error:  0.5800691666350913  &&& Average Error: 0.2906991698677754
Error:: of Iteration 661 =>  Maximum Error:  0.5800317964496984  &&& Average

Error:: of Iteration 732 =>  Maximum Error:  0.5774435910307364  &&& Average Error: 0.2906213518410793
Error:: of Iteration 733 =>  Maximum Error:  0.5774042465911743  &&& Average Error: 0.2906204171230425
Error:: of Iteration 734 =>  Maximum Error:  0.577364715125942  &&& Average Error: 0.2906194871538291
Error:: of Iteration 735 =>  Maximum Error:  0.5773249956789536  &&& Average Error: 0.2906185618434726
Error:: of Iteration 736 =>  Maximum Error:  0.5772850876166975  &&& Average Error: 0.2906176410776831
Error:: of Iteration 737 =>  Maximum Error:  0.5772449906629192  &&& Average Error: 0.2906167247245391
Error:: of Iteration 738 =>  Maximum Error:  0.5772047049421453  &&& Average Error: 0.2906158126407923
Error:: of Iteration 739 =>  Maximum Error:  0.5771642310300261  &&& Average Error: 0.2906149046773912
Error:: of Iteration 740 =>  Maximum Error:  0.5771235700077539  &&& Average Error: 0.2906140006839331
Error:: of Iteration 741 =>  Maximum Error:  0.5770827235173498  &&& Avera

Error:: of Iteration 812 =>  Maximum Error:  0.5740398847235664  &&& Average Error: 0.29055666554670745
Error:: of Iteration 813 =>  Maximum Error:  0.5739968727037582  &&& Average Error: 0.29055598700251806
Error:: of Iteration 814 =>  Maximum Error:  0.5739538403845416  &&& Average Error: 0.2905553115403192
Error:: of Iteration 815 =>  Maximum Error:  0.5739107869990184  &&& Average Error: 0.2905546391852251
Error:: of Iteration 816 =>  Maximum Error:  0.5738677118538522  &&& Average Error: 0.2905539699643784
Error:: of Iteration 817 =>  Maximum Error:  0.5738246143328971  &&& Average Error: 0.29055330390669126
Error:: of Iteration 818 =>  Maximum Error:  0.5737814939006715  &&& Average Error: 0.29055264104257494
Error:: of Iteration 819 =>  Maximum Error:  0.5737383501056232  &&& Average Error: 0.2905519814036539
Error:: of Iteration 820 =>  Maximum Error:  0.5736951825830832  &&& Average Error: 0.29055132502246445
Error:: of Iteration 821 =>  Maximum Error:  0.5736519910578362  &&&

Error:: of Iteration 893 =>  Maximum Error:  0.5705143542734611  &&& Average Error: 0.29051130730855595
Error:: of Iteration 894 =>  Maximum Error:  0.5704705747590628  &&& Average Error: 0.29051082003990053
Error:: of Iteration 895 =>  Maximum Error:  0.5704267744682106  &&& Average Error: 0.29051033325870385
Error:: of Iteration 896 =>  Maximum Error:  0.5703829522129138  &&& Average Error: 0.2905098469439711
Error:: of Iteration 897 =>  Maximum Error:  0.5703391067996421  &&& Average Error: 0.2905093610746294
Error:: of Iteration 898 =>  Maximum Error:  0.5702952370288937  &&& Average Error: 0.2905088756293038
Error:: of Iteration 899 =>  Maximum Error:  0.570251341694381  &&& Average Error: 0.2905083905860975
Error:: of Iteration 900 =>  Maximum Error:  0.5702074195818069  &&& Average Error: 0.2905079059223727
Error:: of Iteration 901 =>  Maximum Error:  0.5701634694672292  &&& Average Error: 0.29050742161453985
Error:: of Iteration 902 =>  Maximum Error:  0.570119490115011  &&& Av

Error:: of Iteration 973 =>  Maximum Error:  0.5668302081346208  &&& Average Error: 0.2904732143306467
Error:: of Iteration 974 =>  Maximum Error:  0.5667817348870288  &&& Average Error: 0.2904727875903242
Error:: of Iteration 975 =>  Maximum Error:  0.5667332727401636  &&& Average Error: 0.2904723619644179
Error:: of Iteration 976 =>  Maximum Error:  0.5666848238779789  &&& Average Error: 0.29047193735130217
Error:: of Iteration 977 =>  Maximum Error:  0.5666363902606316  &&& Average Error: 0.29047151365347484
Error:: of Iteration 978 =>  Maximum Error:  0.566587973635663  &&& Average Error: 0.29047109077769434
Error:: of Iteration 979 =>  Maximum Error:  0.5665395755496425  &&& Average Error: 0.29047066863504895
Error:: of Iteration 980 =>  Maximum Error:  0.5664911973600342  &&& Average Error: 0.290470247140968
Error:: of Iteration 981 =>  Maximum Error:  0.5664428402470865  &&& Average Error: 0.29046982621518996
Error:: of Iteration 982 =>  Maximum Error:  0.5663945052255647  &&& A

In [223]:
# Averaging
sum_H = np.zeros(H[0].shape);
for h in H:
    sum_H = sum_H + h;
H_star = sum_H/len(H);  # <=======================   output -  H*


In [225]:
H_star.shape # (reduced dimension x sample number )

(10, 1474)